In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt


def image_to_coe_rgb(image_path, output_file):
    img = Image.open(image_path).convert('RGB') 
    print(f"Image size: {img.size}")
    assert img.size == (512, 512), "The image must be 512x512 pixels."

    # Split the image into R, G, B channels
    r, g, b = img.split()

    # Convert each channel to a numpy array
    r_array = np.array(r, dtype=np.uint8)
    g_array = np.array(g, dtype=np.uint8)
    b_array = np.array(b, dtype=np.uint8)

    # Stack the channels side by side to create the new image
    entire_input_image = np.hstack((r_array, g_array, b_array))

    # Display the new image
    plt.imshow(entire_input_image)
    plt.title("Entire Input Image (RGB Channels Stacked Side by Side)")
    plt.axis('off')
    plt.show()

    # Write the .coe file
    with open(output_file, 'w') as f:
        f.write("memory_initialization_radix=10;\n")
        f.write("memory_initialization_vector=\n")
        
        # Flatten the stacked image array and write pixel values in unsigned decimal
        decimal_values = [f"{pixel}" for pixel in entire_input_image.flatten()]
        f.write(",\n".join(decimal_values) + ";\n")

    print(f"COE file '{output_file}' created successfully.")
    return entire_input_image

image_path = 'lena_color.gif'
output_file = 'lena_color_processed.coe'
entire_input_image = image_to_coe_rgb(image_path, output_file)
# 786_432 elements in BRAM


def verify_coe_file(output_file, width, height):
    with open(output_file, 'r') as f:
        lines = f.readlines()
    pixel_values = [int(value.strip()) for value in ''.join(lines[2:]).split(',') if value.strip().isdigit()]
    reconstructed_image = np.array(pixel_values, dtype=np.uint8).reshape((height, width))

    plt.imshow(reconstructed_image, cmap='gray')
    plt.title("Reconstructed Image from COE File")
    plt.axis('off')
    plt.show()
    return reconstructed_image

reconstructed_image = verify_coe_file(output_file, 3 * 512, 512)

assert np.array_equal(entire_input_image, reconstructed_image), "Reconstructed image does not match the original!"
print("Verification successful: The reconstructed image matches the original.")

In [ ]:
def crop_image_to_512(image_path, output_path):
    img = Image.open(image_path)
    cropped_img = img.crop((0, 0, 512, 512))  # Crop to 512x512
    cropped_img.save(output_path)
    print(f"Cropped image saved to {output_path}")

cropped_image_path = "cropped_image_512x512.jpg"
crop_image_to_512(image_path, cropped_image_path)

In [ ]:
import re

def read_numbers_from_file(filename):
    """Read the text file and extract numbers"""
    numbers = []
    with open(filename, 'r') as file:
        for line in file:
            # Extract numbers using regex
            extracted = re.findall(r'-?\d+', line)
            # Convert to integers and clip to 8-bit unsigned range (0-255)
            processed = [max(0, min(255, int(num))) for num in extracted]
            numbers.extend(processed)
    return numbers


def write_coe_file(numbers, coe_filename):
    """Write numbers to a .coe file for BRAM initialization in Vivado"""
    with open(coe_filename, 'w') as coe_file:
        coe_file.write("memory_initialization_radix=10;\n")
        coe_file.write("memory_initialization_vector=\n")
        coe_file.write("\n".join(map(str, numbers)) + ";")


def main():
    input_filename = "input.txt"  
    output_filename = "bram_data_to_TOP_v2.coe"

    numbers = read_numbers_from_file(input_filename)
    total_numbers = len(numbers)

    print(f"Total numbers extracted: {total_numbers}")

    write_coe_file(numbers, output_filename)
    print(f"COE file '{output_filename}' created successfully!")

if __name__ == "__main__":
    main()

Total numbers extracted: 786432
COE file 'bram_data_to_TOP.coe' created successfully!
